# Profitable App Profiles for the App Store and Google Play Markets

The aim of this project is to find apps that are profitable for App Store and Google Play Store by analyzing data and helping developers to make data driven decisions with respect to the kind of apps they build.

## Exploring Data

Collecting data requires significant amount of time and energy. So we must always try to find relevant existing data.Luckily, there are two datasets that are suitable for our purpose.These datasets are available on Kaggle.

- [A data set](https://www.kaggle.com/lava18/google-play-store-apps/home) containing data about approximately ten thousand Android apps from Google Play.

- [A data set](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home) containing data about approximately seven thousand iOS apps from the App Store.

Let's start by opening the two datasets

In [2]:
from csv import reader
# AppleStore data #
opened_file = open("AppleStore.csv")
read_file = reader(opened_file)
apple_Store = list(read_file)
apple_Store_header = apple_Store[0]
apple_Store = apple_Store[1:]

# googleplayStore data #
opened_file = open("googleplaystore.csv")
read_file = reader(opened_file)
google_Store = list(read_file)
google_Store_header = google_Store[0]
google_Store = google_Store[1:]




In [3]:
# exploring datasets #
def explore_data(dataset, start, end, rows_and_cols = False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n')
    
    if rows_and_cols == True:
        print("Number of rows:", len(dataset))
        print("Number of columns:", len(dataset[0]))
        


In [4]:
print(apple_Store_header)
print("\n")
explore_data(apple_Store, 1, 5, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']


Number of rows: 7197
Number of columns: 16


In [4]:
print(google_Store_header)
print("\n")
explore_data(google_Store, 1, 5, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


Number of rows: 10841
Number of columns: 13


## Deleting Incorrect Data

The google play store dataset has a row with incorrect data where the row with index number 10472 corresponding to the app "Life Made WI-Fi Touchscreen Photo Frame" has a rating of 19. This is clearly off because the maximum rating for a Google Play app is 5. As a consequence, we'll delete this row.

In [5]:
print(google_Store[10472])

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


In [7]:
del(google_Store[10472])

In [8]:
print(len(google_Store))

10840


## Removing Duplicate Data

If we explore the google play store dataset, we can observe that for some apps there are duplicate entries. For example, Instagram has four entries.

In [9]:
for app in google_Store:
    name = app[0]
    if name == 'Instagram':
        print(app)


['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


There are totally 1181 apps that occur more than once. If we observe the duplicate entries of the Instagram app closely, they differ only in the number of reviews. We can delete duplicate entries randomly.But, a better way is to come up with a criterion for deletion. One such criterion is to keep the entry with highest number of reviews and remove the other entries for any given app. This criterion is based on the fact that higher the number of reviews, more reliable the data.

In [10]:
unique_apps = []
duplicate_apps = []
for app in google_Store:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
print("Number of duplicate apps:",len(duplicate_apps))
    

Number of duplicate apps: 1181


We start by constructing a dictionary where each key is a app name and the corresponding value is the highest number of reviews for that app.

In [11]:
reviews_max = {}

for app in google_Store:
    name = app[0]
    n_reviews = float(app[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
print(len(reviews_max))    

9659


We remove the duplicate entries by using the dicitonary created above.
- We start by initializing two empty lists android_clean which will store our cleaned data and already_added which just stores the app names.
- We loop over the google store dataset and check if the number of reviews for an app matches the number of reviews for that app in the dicitionary. 
- We also check whether the name of the app is already present in the list already_added. we add this additional check because there can be multiple duplicate app entries having the same value for number of reviews.

In [12]:
android_clean = []
already_added = []

for app in google_Store:
    name = app[0]
    n_reviews = float(app[3])
    
    if (reviews_max[name] == n_reviews) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name)

explore_data(android_clean,1,5, True)
        

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


Number of rows: 9659
Number of columns: 13


## Removing Non-English apps

In this project, we'd like to analyze only the apps that are directed towards English-speaking audience.So in our next step we are going to remove the apps that are not directed towards English-speaking audience.One way to go about removing these apps is to use the built-in function ord() to check whether the numbers corresponding to characters in the app name are all in the range 0 to 127.If an app name contains a character that is greater than 127, then it probably means that the app has a non-English name.But, some of the english apps may contain emojis which fall outside the ASCII range and removing them will result in loss of useful data.Hence we will remove the app only if it's name has more than three ASCII characters.

In [13]:
def is_english(string):
    non_english_count = 0
    for char in string:
        if ord(char) > 127: 
            non_english_count += 1
        if non_english_count > 3:
            return False
    return True

print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

True
False
True
True


In [14]:
# Filtering non-english apps from the AppStore dataset #

apple_english_apps = []
for app in apple_Store:
    name = app[1]
    english_app = is_english(name)
    if english_app == True:
        apple_english_apps.append(app)

# Filtering non-english apps from the GoogleStore dataset #

google_english_apps = []
for app in android_clean:
    name = app[0]
    english_app = is_english(name)
    if english_app == True:
        google_english_apps.append(app)

explore_data(apple_english_apps,0,5,True)
print('\n')
explore_data(google_english_apps,0,5,True)
    

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']


Number of rows: 6183
Number of columns: 16


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 

## Isolating Free Apps

In this project, we are going to analyze only the apps that are free. So in our next step we are going to isolate free and non-free apps.

In [15]:
# isolating free apps in AppStore dataset #
apple_free_english_apps = []
for app in apple_english_apps:
    price = app[4]
    if price == '0.0':
        apple_free_english_apps.append(app)

# isolating free apps in GoogleStore dataset #
google_free_english_apps = []
for app in google_english_apps:
    price = (app[7])
    if price == '0':
        google_free_english_apps.append(app)
        

explore_data(apple_free_english_apps,0,5,True)
print('\n')
explore_data(google_free_english_apps,0,5,True)
        
    
    

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']


Number of rows: 3222
Number of columns: 16


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 

## Most common apps by genre

Our aim is to determine what kinds of apps are likely to attract more users and to minimize risks and overhead our validation strategy for an app idea is comprised of three steps:

- Build a minimal android version of the app, and add it to Google Play.

- If the app has a good response from users, we develop it further.

- If the app is profitable afer six months, we build an iOS version of the app and add it to the App Store.


Let's begin our analysis by getting a sense of what are the most common genres for AppStore and Google Play Store. For this, we will build frequency table for the prime genre column for the AppStore dataset and genre column and category column for the Google Play Store dataset.

In [17]:
# function for generating frequency table #

def freq_table(dataset, index):
    freq_dict = {}
    total = 0
    for app in dataset:
        col = app[index]
        total += 1
        if col in freq_dict:
            freq_dict[col] += 1
        else:
            freq_dict[col] = 1
    table_percentages = {}
    for key in freq_dict:
        percentage = (freq_dict[key]/total) * 100
        table_percentages[key] = percentage
    return table_percentages
            

In [18]:
# function for displaying frequency tables #

def display_table(dataset,index):
    table = freq_table(dataset,index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
    
    sorted_table = sorted(table_display, reverse = True)
    for val in sorted_table:
        print(val[1], ":", val[0])



Let's start by examining the frequency table of the prime genre column in the AppStore dataset. We can see that almost half of the apps (58.16%) are games.Entertainment apps are close to 8%, followed by Photo & Video apps which are close to 5% apps. Only 3.66% apps are designed for Education followed by Social Networking(3.29%).

The general impression is that AppStore is dominated by apps designed for entertainment(Games,Entertainment,Photo & Video,Social Networking) when compared to apps designed for practical purposes(Eucation,Finance,Book). The presence of large number of entertainment apps does not guarantee large number of users.

In [19]:
display_table(apple_free_english_apps,11) # prime_genre

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


Let's continue examining the frequency tables for the category and genres column in GoogleStore dataset. We can observe that there is a good balance between the apps designed for practical purposes(Family,Tools,Business,Lifestyle) and apps designed for entertainment purposes which is also confirmed by the genres frequency table.

In [20]:
display_table(google_free_english_apps,1) # category

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

In [21]:
display_table(google_free_english_apps,9) # Genres

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

The difference between category and genres column is not clear. However, category column has more granular representation of categories. So going forward, we will analyze only the category column.

Up to this point, we have observed that AppStore is dominated by Fun apps whereas GoogleStore has a balanced landscape between Fun apps and Practical apps.

## Most Popular Apps by Genre on the App Store

One way to find out what genres are most popular(having the most users) is to calculate the average number of installs per each app genre. For the App Store dataset this information is missing. As a workaround, we'll take the total number of user ratings as a proxy.

Let's calculate the average number of user reviews per genre on the AppStore.

In [22]:
genres = freq_table(apple_free_english_apps,11)

for genre in genres:
    total = 0
    len_genre = 0
    for app in apple_free_english_apps:
        genre_app = app[11]
        if genre_app == genre:
            no_of_user_ratings = float(app[5])
            total += no_of_user_ratings
            len_genre += 1
    avg_ratings = total/len_genre
    print(genre, ":", avg_ratings)
            

Finance : 31467.944444444445
Reference : 74942.11111111111
Productivity : 21028.410714285714
Travel : 28243.8
Photo & Video : 28441.54375
Health & Fitness : 23298.015384615384
Weather : 52279.892857142855
Lifestyle : 16485.764705882353
Book : 39758.5
Navigation : 86090.33333333333
Games : 22788.6696905016
News : 21248.023255813954
Social Networking : 71548.34905660378
Utilities : 18684.456790123455
Business : 7491.117647058823
Catalogs : 4004.0
Music : 57326.530303030304
Entertainment : 14029.830708661417
Medical : 612.0
Sports : 23008.898550724636
Shopping : 26919.690476190477
Education : 7003.983050847458
Food & Drink : 33333.92307692308


On an average, Navigation apps have the highest number of user reviews but, this number is heavily influenced by Waze and Google Maps which have close to half a million user reviews together. 

In [23]:
for app in apple_free_english_apps:
    genre = app[11]
    if genre == "Navigation":
        print(app[1], ":", app[5])


Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


The same pattern applies to Social Networking apps where the average number is heavily influenced by Facebook, WhatsApp, Skype etc.

In [24]:
for app in apple_free_english_apps:
    genre = app[11]
    if genre == 'Social Networking':
        print(app[1], ":" ,app[5])

Facebook : 2974676
Pinterest : 1061624
Skype for iPhone : 373519
Messenger : 351466
Tumblr : 334293
WhatsApp Messenger : 287589
Kik : 260965
ooVoo – Free Video Call, Text and Voice : 177501
TextNow - Unlimited Text + Calls : 164963
Viber Messenger – Text & Call : 164249
Followers - Social Analytics For Instagram : 112778
MeetMe - Chat and Meet New People : 97072
We Heart It - Fashion, wallpapers, quotes, tattoos : 90414
InsTrack for Instagram - Analytics Plus More : 85535
Tango - Free Video Call, Voice and Chat : 75412
LinkedIn : 71856
Match™ - #1 Dating App. : 60659
Skype for iPad : 60163
POF - Best Dating App for Conversations : 52642
Timehop : 49510
Find My Family, Friends & iPhone - Life360 Locator : 43877
Whisper - Share, Express, Meet : 39819
Hangouts : 36404
LINE PLAY - Your Avatar World : 34677
WeChat : 34584
Badoo - Meet New People, Chat, Socialize. : 34428
Followers + for Instagram - Follower Analytics : 28633
GroupMe : 28260
Marco Polo Video Walkie Talkie : 27662
Miitomo : 2

Reference Apps have 74,942 ratings.But,the number is heavily influenced by Bible and Dictionary.com. However, this genre seems to show some potential.

In [25]:
for app in apple_free_english_apps:
    genre = app[11]
    if genre == 'Reference':
        print(app[1], ":", app[5] )

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


Our aim is to find popular genres, but navigation and social networking apps seem more popular than they actually are. The average ratings is heavily influenced by few apps and other apps struggle to get past the threshold. One thing we can do is to turn a popular book or a holy book into an app. We can add new features in addition to the raw version of the book. The features can be daily quotes from the book, quizzes related to content of the book etc. We can also embed a dictionary within the app, so that the users don't need to exit our app to look for the meanings on web. 


This idea seems to go well with the fact that AppStore is dominated by fun apps. The market might be saturated for fun apps and there is a high chance that an app designed for practical purposes might stand out among the huge number of apps on the AppStore.

## Most Popular Apps by Genre on Google Play

For the Google Play market, we have data about the number of installs.So, we should be able to get a clear picture about genre 
popularity. However, the install numbers don't seem precise enough - we can see that most values are open-ended(100+, 1000+ etc)

In [26]:
display_table(google_free_english_apps,5)

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


For instance, we don't know whether an app with 100,000+ installs has 100,000 installs or 500,000. However, we don't need precise data for our analyis. We are going to leave the numbers as they are , which means that we will consider that an app with 100,000+ installs has 100,000 installs and an app with 500,000+ installs has 500,000 installs. To perform computations, we will need to remove the commas and the plus characters, otherwise the conversion will fail and raise an error.

In [27]:
Category = freq_table(google_free_english_apps, 1)

for genre in Category:
    total = 0
    len_category = 0
    for app in google_free_english_apps:
        category_app = app[1]
        if category_app == genre:
            installs = app[5]
            installs = installs.replace("+", "")
            installs = installs.replace(",", "")
            installs = float(installs)
            total += installs
            len_category += 1
    average_num_installs = total/len_category
    print(genre, ":", average_num_installs)

FINANCE : 1387692.475609756
GAME : 15588015.603248259
PHOTOGRAPHY : 17840110.40229885
WEATHER : 5074486.197183099
BEAUTY : 513151.88679245283
MAPS_AND_NAVIGATION : 4056941.7741935486
VIDEO_PLAYERS : 24727872.452830188
MEDICAL : 120550.61980830671
DATING : 854028.8303030303
FAMILY : 3695641.8198090694
SHOPPING : 7036877.311557789
HEALTH_AND_FITNESS : 4188821.9853479853
LIFESTYLE : 1437816.2687861272
COMMUNICATION : 38456119.167247385
AUTO_AND_VEHICLES : 647317.8170731707
PRODUCTIVITY : 16787331.344927534
FOOD_AND_DRINK : 1924897.7363636363
PARENTING : 542603.6206896552
EVENTS : 253542.22222222222
SOCIAL : 23253652.127118643
BOOKS_AND_REFERENCE : 8767811.894736841
HOUSE_AND_HOME : 1331540.5616438356
TRAVEL_AND_LOCAL : 13984077.710144928
NEWS_AND_MAGAZINES : 9549178.467741935
TOOLS : 10801391.298666667
ENTERTAINMENT : 11640705.88235294
LIBRARIES_AND_DEMO : 638503.734939759
COMICS : 817657.2727272727
ART_AND_DESIGN : 1986335.0877192982
PERSONALIZATION : 5201482.6122448975
EDUCATION : 18334

On an average, Communication apps have the highest number of installs and this figure is heavily influenced by WhatsApp,Skype, Messenger, Google Chrome etc.

In [30]:
for app in google_free_english_apps:
    category = app[1]
    if category == 'COMMUNICATION':
        print(app[0], ":", app[5])
    

WhatsApp Messenger : 1,000,000,000+
Messenger for SMS : 10,000,000+
My Tele2 : 5,000,000+
imo beta free calls and text : 100,000,000+
Contacts : 50,000,000+
Call Free – Free Call : 5,000,000+
Web Browser & Explorer : 5,000,000+
Browser 4G : 10,000,000+
MegaFon Dashboard : 10,000,000+
ZenUI Dialer & Contacts : 10,000,000+
Cricket Visual Voicemail : 10,000,000+
TracFone My Account : 1,000,000+
Xperia Link™ : 10,000,000+
TouchPal Keyboard - Fun Emoji & Android Keyboard : 10,000,000+
Skype Lite - Free Video Call & Chat : 5,000,000+
My magenta : 1,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Seznam.cz : 1,000,000+
Antillean Gold Telegram (original version) : 100,000+
AT&T Visual Voicemail : 10,000,000+
GMX Mail : 10,000,000+
Omlet Chat : 10,000,000+
My Vodacom SA : 5,000,000+
Microsoft Edge : 5,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Calls & Text by Mo+ : 5,000,000+
free 

The games genre seems to be fairly popular as well. But, as per our previous analysis the market for fun apps is a bit saturated. Our aim is to recommend an app genre that shows potential of being profitable on both the App Store and Google Play. So we would like to come up with a different app profile recommendation.

In [31]:
for app in google_free_english_apps:
    category = app[1]
    if category == "GAME":
        print(app[0], ":", app[5])

Solitaire : 10,000,000+
Sonic Dash : 100,000,000+
PAC-MAN : 100,000,000+
Bubble Witch 3 Saga : 50,000,000+
Race the Traffic Moto : 10,000,000+
Marble - Temple Quest : 10,000,000+
Shooting King : 10,000,000+
Geometry Dash World : 10,000,000+
Jungle Marble Blast : 5,000,000+
Roll the Ball® - slide puzzle : 100,000,000+
Block Craft 3D: Building Simulator Games For Free : 50,000,000+
Farm Fruit Pop: Party Time : 1,000,000+
Love Balls : 50,000,000+
Piano Tiles 2™ : 100,000,000+
Pokémon GO : 100,000,000+
Paint Hit : 10,000,000+
Snake VS Block : 50,000,000+
Rolly Vortex : 10,000,000+
Woody Puzzle : 1,000,000+
Stack Jump : 10,000,000+
The Cube : 5,000,000+
Extreme Car Driving Simulator : 100,000,000+
Bricks n Balls : 1,000,000+
The Fish Master! : 1,000,000+
Color Road : 10,000,000+
Draw In : 10,000,000+
PLANK! : 500,000+
Looper! : 1,000,000+
Trivia Crack : 100,000,000+
Will it Crush? : 5,000,000+
Tomb of the Mask : 5,000,000+
Baseball Boy! : 10,000,000+
Hello Stars : 10,000,000+
Tank Stars : 1

The book and reference genre is quite popular as well. Let's explore this genre in more detail since this idea seems to fit well with our recommendation for the AppStore.

In [33]:
for app in google_free_english_apps:
    category = app[1]
    if category == 'BOOKS_AND_REFERENCE':
        print(app[0], ":", app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

The books and reference genre seems to have a wide variety of apps ranging from dictionaries to programming tutorials and there are very few popular apps that influence the average. This genre shows potential. 

We can observe that there are quite few apps that are built on Holy Books like Quran and Bible, suggesting that building an app around a popular or holy book can be profitable. It seems that taking a popular book and turning it into an app is profitable on both AppStore and Google Play.

## Conclusions

In this Project, we analyzed data about AppStore and Google Play mobile apps with the aim of recommending an app profile that is profitable for both the markets.

We concluded that turning a popular book or a holy book into an app would be profitable for both AppStore and Google Play. Since the market is already full of libraries, we need to add new features in addition to the raw version of the book like daily quotes from the book, quizzes related to the content of the book etc.